In [21]:
import sys
sys.path.insert(0, "/home/tumay/code/data-context-and-setup")

In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# `Logit` on Orders - Logistic Regression (~1h)

## Select features

🎯 Haydi `wait_time` ve `delay_vs_expected` değişkenlerinin çok `iyi/kötü review`lar üzerindeki etkisini inceleyelim.

👉 `orders` training_set’imizi kullanarak iki adet `multivariate logistic regression` çalıştıracağız:
- `logit_one` → `dim_is_one_star` tahmini için  
- `logit_five` → `dim_is_five_star` tahmini için.

 

In [23]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

👉 Dataset’inizi import edin:

In [24]:
from olist.order import Order
orders = Order().get_training_data(with_distance_seller_customer=True)

👉 Kullanmak istediğiniz feature’ları bir listede seçin:

⚠️ Data leakage yaratmadığınızdan emin olun (yani target’tan türetilmiş feature’ları seçmeyin)

💡 `wait_time` ve `delay_vs_expected` değişkenlerinin etkisini anlayabilmek için diğer feature’ların etkisini kontrol etmemiz gerekir, bu yüzden listenize ilgili olabilecek tüm feature’ları dahil edin.

In [25]:
orders.head()

,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status,dim_is_five_star,dim_is_one_star,review_score,number_of_items,number_of_sellers,price,freight_value,distance_seller_customer
0,e481f51cbdc54678b7cc49136f2d6af7,8.436574,15.544063,0.0,delivered,0,0,4,1,1,29.99,8.72,18.063837
1,53cdb2fc8bc7dce0b6741e2150273451,13.782037,19.137766,0.0,delivered,0,0,4,1,1,118.70,22.76,856.292580
2,47770eb9100c2d0c44946d9cf07ec65d,9.394213,26.639711,0.0,delivered,1,0,5,1,1,159.90,19.22,514.130333
3,949d5b44dbf5de918fe9c16f97b45f8a,13.208750,26.188819,0.0,delivered,1,0,5,1,1,45.00,27.20,1822.800366
4,ad21c59c0840e6cb83a9ceb5573f8159,2.873877,12.112049,0.0,delivered,1,0,5,1,1,19.90,8.72,30.174037


In [29]:
selected_features = [
    "wait_time",
    "delay_vs_expected",
    "number_of_sellers",
    "distance_seller_customer",
    "price",
    "freight_value"
    ]


🕵🏻 Feature’larınızın `multicollinearity` durumunu `VIF index` kullanarak kontrol edin.

* Çok yüksek olmamalıdır (tercihen < 10), böylece partial regression coefficient’larına ve ilgili `p-values` değerlerine güvenebiliriz.
* Verinizi standardize etmeyi unutmayın!
    * Bir `VIF Analysis`, bir feature’ın diğer feature’lara karşı regresyonunu yaparak hesaplanır...
    * Bu yüzden herhangi bir linear regression çalıştırmadan önce feature’ların `scale etkisini kaldırmak` ve eşit öneme sahip olmalarını sağlamak istersiniz!
    
    
📚 <a href="https://www.statisticshowto.com/variance-inflation-factor/">Statistics How To - Variance Inflation Factor</a>

📚  <a href="https://online.stat.psu.edu/stat462/node/180/">PennState - Detecting Multicollinearity Using Variance Inflation Factors</a>

⚖️ Standardize etme:

In [30]:
# Standartlaştırma

orders_standardized = orders.copy()

mu = orders[selected_features].mean()
sigma = orders[selected_features].std()
orders_standardized[selected_features] = (orders[selected_features] - mu) / sigma

orders_standardized




,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status,dim_is_five_star,dim_is_one_star,review_score,number_of_items,number_of_sellers,price,freight_value,distance_seller_customer
0,e481f51cbdc54678b7cc49136f2d6af7,-0.431192,15.544063,-0.161781,delivered,0,0,4,1,-0.112544,-0.513802,-0.652038,-0.979475
1,53cdb2fc8bc7dce0b6741e2150273451,0.134174,19.137766,-0.161781,delivered,0,0,4,1,-0.112544,-0.086640,0.000467,0.429743
2,47770eb9100c2d0c44946d9cf07ec65d,-0.329907,26.639711,-0.161781,delivered,1,0,5,1,-0.112544,0.111748,-0.164053,-0.145495
3,949d5b44dbf5de918fe9c16f97b45f8a,0.073540,26.188819,-0.161781,delivered,1,0,5,1,-0.112544,-0.441525,0.206815,2.054621
4,ad21c59c0840e6cb83a9ceb5573f8159,-1.019535,12.112049,-0.161781,delivered,1,0,5,1,-0.112544,-0.562388,-0.652038,-0.959115
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95875,9c5dedf39a927c1b2549525ed64a053c,-0.454309,18.587442,-0.161781,delivered,1,0,5,1,-0.112544,-0.311513,-0.449408,-0.893033
95876,63943bddc261676b46f01ca7ac2f7bd8,1.023841,23.459051,-0.161781,delivered,0,0,4,1,-0.112544,0.183977,-0.123156,-0.212797
95877,83c1379a015df1e13d02aae0204711ab,1.305780,30.384225,-0.161781,delivered,1,0,5,1,-0.112544,0.333684,1.964490,0.617630
95878,11c177c8e97725db2631073c19f07b62,0.483664,37.105243,-0.161781,delivered,0,0,2,2,-0.112544,1.075186,2.715522,-0.387558


👉 Olası multicollinearity durumlarını analiz etmek için VIF Analysis’inizi çalıştırın:

In [31]:
X_standardized = orders_standardized[selected_features]
X_standardized.head()

,wait_time,delay_vs_expected,number_of_sellers,distance_seller_customer,price,freight_value
0,-0.431192,-0.161781,-0.112544,-0.979475,-0.513802,-0.652038
1,0.134174,-0.161781,-0.112544,0.429743,-0.086640,0.000467
2,-0.329907,-0.161781,-0.112544,-0.145495,0.111748,-0.164053
3,0.073540,-0.161781,-0.112544,2.054621,-0.441525,0.206815
4,-1.019535,-0.161781,-0.112544,-0.959115,-0.562388,-0.652038


## Logistic Regressions

👉 İki adet `Logistic Regression` modeli fit edin:
- `logit_one` → `dim_is_one_star` tahmini için
- `logit_five` → `dim_is_five_star` tahmini için.

`Logit 1️⃣`

In [32]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

df = pd.DataFrame()
df["features"] = X_standardized.columns
df["vif_index"] = [vif(X_standardized.values, i) for i in range(X_standardized.shape[1])]
round(df,2)

,features,vif_index
0,wait_time,2.62
1,delay_vs_expected,2.21
2,number_of_sellers,1.02
3,distance_seller_customer,1.41
4,price,1.21
5,freight_value,1.36


`Logit 5️⃣`

In [33]:
logit_one = smf.logit(formula='dim_is_one_star ~'+ ' + '.join(selected_features), data=orders_standardized).fit()
logit_one.summary()

Optimization terminated successfully.
         Current function value: 0.276470
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:        dim_is_one_star   No. Observations:                95872
Model:                          Logit   Df Residuals:                    95865
Method:                           MLE   Df Model:                            6
Date:                Tue, 30 Dec 2025   Pseudo R-squ.:                  0.1357
Time:                        11:33:10   Log-Likelihood:                -26506.
converged:                       True   LL-Null:                       -30669.
Covariance Type:            nonrobust   LLR p-value:                     0.000
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -2.4472      0.013   -191.764      0.000      -2.472      -2.422
wait_time                    0.6907      0.017     41.192      0.000       0.658       0.724
delay_vs_expected            0.2626      0.018     14.238      0.000       0.226       0.299
number_of_sellers            0.2295      0.008     30.500      0.000       0.215       0.244
distance_seller_customer    -0.2193      0.014    -16.023      0.000      -0.246      -0.192
price                        0.0407      0.011      3.776      0.000       0.020       0.062
freight_value                0.1090      0.011      9.790      0.000       0.087       0.131
============================================================================================
"""

💡 Şimdi bu iki logistic regression’ın sonuçlarını analiz etme zamanı:

- Partial coefficient’ları kendi kelimelerinizle yorumlayın.
- `p-values` kullanarak istatistiksel anlamlılıklarını kontrol edin.
- Coefficient önemleri açısından `logit_one` ve `logit_five` arasında herhangi bir fark görüyor musunuz?

In [34]:
# Aşağıdaki cümlelerden doğru olanları aşağıdaki listeye kaydedin.

a = "delay_vs_expected influences five_star ratings even more than one_star ratings"
b = "wait_time influences five_star ratings even more than one_star"

your_answer = []

In [35]:
your_answer = [a]

🧪 __Kodunu Test Et__

In [36]:
from nbresult import ChallengeResult

result = ChallengeResult('logit',
    answers = your_answer
)
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /home/tumay/.pyenv/versions/workintech/bin/python
cachedir: .pytest_cache
rootdir: /home/tumay/code/data-logit/tests
plugins: anyio-4.8.0, typeguard-4.4.2
collecting ... collected 1 item

test_logit.py::TestLogit::test_question PASSED                           [100%]

============================== 1 passed in 0.01s ===============================


💯 You can commit your code:

git add tests/logit.pickle

git commit -m 'Completed logit step'

git push origin master



<details>
    <summary>- <i>Açıklamalar ve ileri seviye kavramlar</i> -</summary>


> _Diğer tüm şeyler sabitken, `delay factor`, 1-yıldız review alma ihtimalini etkilemesinden bile daha fazla, 5-yıldızdan mahrum kalma ihtimalini artırma eğilimindedir. Muhtemelen bunun sebebi, 1-yıldız review’ların bizzat çok kötü ürünleri hedeflemesi, kötü teslimatları değil._

❗️ Ancak tamamen titiz olmak için, **iki farklı modelin coefficient’larını karşılaştırırken daha dikkatli olmamız gerekir**, çünkü **benzer popülasyonlara dayanmayabilirler**!
    Burada 2 alt popülasyonumuz var: (1-yıldız verenler ve 5-yıldız verenler) ve bunlar doğaları gereği farklı davranış kalıpları sergileyebilirler. 5-yıldız vermeye daha meyilli “mutlu insanlar”ın, “gecikme” veya “fiyat” söz konusu olduğunda, 1-yıldızı “Lucky-Luke gibi ateşleyen” “huysuz insanlara” göre daha az hassas olmaları gayet mümkün...

</details>



🏁 Tebrikler!

💾 `logit.ipynb` notebook’unuzu commit ve push etmeyi unutmayın!